In [1]:
# csv 를 읽고 parquet 로 변환하는 코드
from faker import Faker
from pathlib import Path
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

fake = Faker('ko_KR') # locale 정보 설정
Faker.seed()

In [118]:
# 100만건 데이터 생성
# 생성에 걸린시간 측정
gen_begin = pd.Timestamp.now()
curr_time = 3600 * 4
repeat = curr_time
sensor_count = 2000
data = {
    'name': [fake.name() for _ in range(repeat)],
    'address': [fake.address() for _ in range(repeat)],
    'email': [fake.email() for _ in range(repeat)],
    'job': [fake.job() for _ in range(repeat)],
    'company': [fake.company() for _ in range(repeat)],
    'ssn': [fake.ssn() for _ in range(repeat)],
    'residence': [fake.city() for _ in range(repeat)],
    'current_location': [fake.city() for _ in range(repeat)],
    'blood_group': [fake.random_element(elements=('A', 'B', 'AB', 'O')) for _ in range(repeat)],
    'website': [fake.url() for _ in range(repeat)],
    'username': [fake.user_name() for _ in range(repeat)],
    'phone_number': [fake.phone_number() for _ in range(repeat)],
    'company_email': [fake.company_email() for _ in range(repeat)],
    'favorite_color': [fake.color_name() for _ in range(repeat)],
    'uuid': [fake.uuid4() for _ in range(repeat)],
}
# 센서 데이터 추가
for s in range(1, sensor_count+1):
    data[f'sensor{s}'] = [fake.random_int(min=0, max=100) for _ in range(repeat)]
gen_curtime = pd.Timestamp.now() - gen_begin

'생성소요시간: 0 days 00:00:50.479110'

In [121]:
display(f"생성소요시간: {gen_curtime.total_seconds()}초")
display(f'생성 데이터 column 갯수 : {len(data)}개')
display(f'생성 데이터 row 갯수 : {len(data["name"])}개')
display(f'임의 센서 데이터 column 갯수 : {sensor_count}개')

'생성소요시간: 50.47911초'

'생성 데이터 column 갯수 : 2015개'

'생성 데이터 row 갯수 : 14400개'

'임의 센서 데이터 column 갯수 : 2000개'

In [125]:
display(data.keys())

dict_keys(['name', 'address', 'email', 'job', 'company', 'ssn', 'residence', 'current_location', 'blood_group', 'website', 'username', 'phone_number', 'company_email', 'favorite_color', 'uuid', 'sensor1', 'sensor2', 'sensor3', 'sensor4', 'sensor5', 'sensor6', 'sensor7', 'sensor8', 'sensor9', 'sensor10', 'sensor11', 'sensor12', 'sensor13', 'sensor14', 'sensor15', 'sensor16', 'sensor17', 'sensor18', 'sensor19', 'sensor20', 'sensor21', 'sensor22', 'sensor23', 'sensor24', 'sensor25', 'sensor26', 'sensor27', 'sensor28', 'sensor29', 'sensor30', 'sensor31', 'sensor32', 'sensor33', 'sensor34', 'sensor35', 'sensor36', 'sensor37', 'sensor38', 'sensor39', 'sensor40', 'sensor41', 'sensor42', 'sensor43', 'sensor44', 'sensor45', 'sensor46', 'sensor47', 'sensor48', 'sensor49', 'sensor50', 'sensor51', 'sensor52', 'sensor53', 'sensor54', 'sensor55', 'sensor56', 'sensor57', 'sensor58', 'sensor59', 'sensor60', 'sensor61', 'sensor62', 'sensor63', 'sensor64', 'sensor65', 'sensor66', 'sensor67', 'sensor68',

In [90]:
def timestamp2ms(timestamp)->str:
    return str(int(timestamp.total_seconds() * 1000)) + ' ms'
def byte2mb(byte)->float:
    return str(round(byte / 1024 / 1024, 2)) + ' MB'

In [122]:
# 테스트 할 항목
# 1. csv 로 저장했을때 용량
# 2. parquet 로 저장했을때 용량
# 3. csv 파일을 읽는데 걸리는 시간
# 4. parquet 파일을 읽는데 걸리는 시간
# 5. csv 파일을 parquet 파일로 변환하는데 걸리는 시간
# 6. parquet 파일을 csv 파일로 변환하는데 걸리는 시간
# 7. csv 파일을 parquet 파일로 변환해서 저장하고, parquet 파일을 읽는데 걸리는 시간 (컬럼의 타입이 없는 경우 읽기속도에 차이가 나는지 확인하기 위함)
# 8. csv 파일의 sensor 컬럼을 변환해서 저장하는데 걸리는 시간
# 9. parquet 파일의 sensor 컬럼을 변환해서 저장하는데 걸리는 시간

save_dir = Path('result')
csv_fn = save_dir/'csv_data.csv'
parquet_fn = save_dir/'parquet.parquet'
cv_csv2parquet_fn = save_dir/'cv_csv2parquet.parquet'
cv_parquet2csv_fn = save_dir/'cv_parquet2csv.csv'
modi_csv_fn = save_dir/'modi_csv_data.csv'
modi_parquet_fn = save_dir/'modi_parquet.parquet'


# 1. dict 를 csv 로 저장
csv_save_begin = pd.Timestamp.now()
df = pd.DataFrame(data)
df.to_csv(csv_fn, index=False)
csv_save_end = pd.Timestamp.now()

# 2. dict 를 parquet 로 저장
parquet_save_begin = pd.Timestamp.now()
table2 = pa.table(data)
pq.write_table(table2, parquet_fn)
parquet_save_end = pd.Timestamp.now()

# 3. 저장된 csv 를 df 로 읽기
csv_read_begin = pd.Timestamp.now()
df = pd.read_csv(csv_fn)
csv_read_end = pd.Timestamp.now()

# 5. csv 를 읽은 df 를 parquet 로 변환하고 저장
cv_csv2parquet_begin = pd.Timestamp.now()
table2 = pa.table(df)
pq.write_table(table2, cv_csv2parquet_fn)
cv_csv2parquet_end = pd.Timestamp.now()

# 4. 저장된 parquet 을 df 로 읽기
parquet_read_begin = pd.Timestamp.now()
table2 = pq.read_table(parquet_fn)
df = table2.to_pandas()
parquet_read_end = pd.Timestamp.now()

# 6. parquet 을 읽은 df 를 csv 로 변환하고 저장
cv_parquet2csv_begin = pd.Timestamp.now()
df.to_csv(cv_parquet2csv_fn, index=False)
cv_parquet2csv_end = pd.Timestamp.now()

# 수정은 sensor 1~ 5 에 100을 나눠 0~1 사이의 값으로 변환
# 7. csv 를 읽고, 수정한 후 csv 로 저장
modi_csv_begin = pd.Timestamp.now()
df = pd.read_csv(csv_fn)
for s in range(1, sensor_count+1):
    df[f'sensor{s}'] = df[f'sensor{s}'] / 100
df.to_csv(modi_csv_fn, index=False)
modi_csv_end = pd.Timestamp.now()

# 8. parquet 을 읽고, 수정한 후 parquet 로 저장
modi_pq_begin = pd.Timestamp.now()
table2 = pq.read_table(parquet_fn)
table2 = table2.to_pandas()
for s in range(1, sensor_count+1):
    table2[f'sensor{s}'] = table2[f'sensor{s}'] / 100

pq.write_table(pa.table(table2), modi_parquet_fn)
modi_pq_end = pd.Timestamp.now()

In [123]:
info_table = pd.Series(data=[len(data.keys()),len(data["name"]),], index=['column', 'row'])
display(info_table)
bench = {
    'csv': {
        '원본저장시간' : round((csv_save_end - csv_save_begin).total_seconds(), 2),
        '원본읽기시간' : round((csv_read_end - csv_read_begin).total_seconds(), 2),
        '변환저장시간' : round((cv_csv2parquet_end - cv_csv2parquet_begin).total_seconds(), 2),
        '수정저장시간' : round((modi_csv_end - modi_csv_begin).total_seconds(), 2),
        '저장용량' : round((csv_fn.stat().st_size / 1024 / 1024), 2),
        '변환용량' : round((cv_csv2parquet_fn.stat().st_size / 1024 / 1024), 2),
    },
    'parquet': {
        '원본저장시간' : round((parquet_save_end - parquet_save_begin).total_seconds(), 2),
        '원본읽기시간' : round((parquet_read_end - parquet_read_begin).total_seconds(), 2),
        '변환저장시간' : round((cv_parquet2csv_end - cv_parquet2csv_begin).total_seconds(), 2),
        '수정저장시간' : round((modi_pq_end - modi_pq_begin).total_seconds(), 2),
        '저장용량' : round((parquet_fn.stat().st_size / 1024 / 1024), 2),
        '변환용량' : round((cv_parquet2csv_fn.stat().st_size / 1024 / 1024), 2),
    }
}
bench_table = pd.DataFrame(bench)
bench_table['diff'] = bench_table['csv'] - bench_table['parquet']
bench_table['unit'] = ['ms', 'ms', 'ms', 'ms', 'MB', 'MB']
bench_table['ratio'] = round(bench_table['csv'] / bench_table['parquet'], 3)

display(bench_table)

column     2015
row       14400
dtype: int64

,csv,parquet,diff,unit,ratio
원본저장시간,6.92,1.51,5.41,ms,4.583
원본읽기시간,1.38,0.14,1.24,ms,9.857
변환저장시간,0.64,3.80,-3.16,ms,0.168
수정저장시간,22.30,1.37,20.93,ms,16.277
저장용량,83.73,27.41,56.32,MB,3.055
변환용량,27.93,83.73,-55.80,MB,0.334


In [11]:
# xx.xxx s 으로 표시하도록 변경
t = csv_save_end - csv_save_begin
print(f'csv 저장: {t.total_seconds():.3f} s')


csv 저장: 0.011 s


In [73]:
# int로 변경
table2.loc[['저장', '읽기', '변환'], table2.columns].astype(int).astype(str).map(lambda x: x + ' ms')


,csv,parquet
저장,11 ms,6 ms
읽기,12 ms,5 ms
변환,6 ms,9 ms
